Para realizar el proceso de ETL, dado que son 3 los archivos que me dieron, decidí hacer un .ipynb para cada uno de estos archivos. Este es el .ipynb dedicado al archivo user_reviews

In [1]:
# Lo primero que hay que hay que hacer es descomprimir el archivo, ya que viene con una compresión .gz (gzip) con la cuál no puedo trabajar usando Python (o al menos no de manera cómoda)
# Importo las librerías necesarias para descomprimir el archivo y luego cargarlo cómo dataframe
import gzip
import ast
import pandas as pd


rutaArchivo = 'rawData/user_reviews.json.gz'

# Lista que va almacenar cada línea del archivo ya descomprimido
data = []

# Metodo open para abrir el archivo comprimido
with gzip.open(rutaArchivo, 'rt', encoding='utf-8') as file:
    # Bucle para iterar el archivo línea por línea
    for line in file:
        try:
            # ast.literal_eval lanza una excepción si la entrada no es un tipo de dato valido en Python. Si lo es, guarda la línea en una variable temporal llamada json_data y luego esa variable temporal es agregada a la lista data previamente creada
            json_data = ast.literal_eval(line)
            data.append(json_data)

        except ValueError as e:
            # Si no se puede leer la línea porque el tipo de dato no es válido en Python, lanza una excepción
            print(f"Error en la línea: {line}")
            continue

# Finalmente, los datos contenidos en la lista data, son alojados en un dataframe de pandas
user_reviews = pd.DataFrame(data)

Primer vistazo del dataset user_reviews descomprimido

In [2]:
user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Dando un primer vistazo y basandome en lo que dice el diccionario de datos; la columna reviews contiene Review de usuario en formato Json, lo mismo que había sucedido con user_items, por lo cuál tengo que desanidar esa columna

In [3]:
user_reviews["reviews"] # Por salida concluyo que cada fila tiene una lista, que en su interior contiene al menos un diccionario

0        [{'funny': '', 'posted': 'Posted November 5, 2...
1        [{'funny': '', 'posted': 'Posted June 24, 2014...
2        [{'funny': '', 'posted': 'Posted February 3.',...
3        [{'funny': '', 'posted': 'Posted October 14, 2...
4        [{'funny': '3 people found this review funny',...
                               ...                        
25794    [{'funny': '', 'posted': 'Posted May 31.', 'la...
25795    [{'funny': '', 'posted': 'Posted June 17.', 'l...
25796    [{'funny': '1 person found this review funny',...
25797    [{'funny': '', 'posted': 'Posted July 21.', 'l...
25798    [{'funny': '1 person found this review funny',...
Name: reviews, Length: 25799, dtype: object

In [4]:
user_reviews["reviews"][0] # Hechando un vistazo a la primera lista, veo que efectivamente, son un conjunto de reviews hechas por el usuario con el "user_id" correspondiente. Es el conjunto de todas las reseñas que escribió el usuario que aparece en esa columna

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [5]:
# Código de desanidación
# Lista que va a almacenar las filas del dataframe final
lista_de_items = []

# Bucle que itera cada fila de la columna reviews del dataframe user_reviews
for i in range(len(user_reviews["reviews"])): # En cada iteración, "user_reviews["reviews"][i]", va a ser una lista que contiene diccionarios, los cuales tienen cómo clave las 7 primeras columnas del dataframe final
    
    
    user_id = user_reviews["user_id"][i]  # 8a columna del dataframe final, sirve para reconocer que usuario hizo la reseña correspondiente
    
    # Bucle que itera cada diccionario dentro de la lista. Cada j es un diccionario y corresponde con una fila en el dataframe final.
    for j in user_reviews["reviews"][i]:  
        
        # Variable que va almacenar los diccionarios de cada iteración
        diccionario_items = j.copy()
        diccionario_items["user_id"] = user_id  # Agregar el user_id al diccionario
        lista_de_items.append(diccionario_items)  # Agregar el diccionario a la lista creada previamente

# Crear un dataframe que almacene los datos de la lista
Reviews_complete = pd.DataFrame(lista_de_items)


In [7]:
# Vistazo del dataset desanidado
Reviews_complete.head(3) # Nota: No agregué el user_url porque me pareció una columna innecesaria para lo que me pidieron, tanto en este dataset, cómo en el de user_items, decidí descartarlas. 

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479


Una vez que ya transformé los datos a un formato más leíble, los subo a la carpeta EDA para hacer el análisis posteriormente, los subo en formato parquet con compresión snappy para optimizar el espacio

In [8]:
Reviews_complete.to_parquet("../EDA/user_reviews_complete.parquet",compression="snappy")